In [344]:
# Dynamic Scrapping

#@author: yagizcanurkmez


In [295]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select

In [296]:
url = 'https://weather.uwyo.edu/upperair/naconf.html'
driver = webdriver.Chrome()
driver.get(url)

In [297]:
table = driver.find_element(By.TAG_NAME,"table")
table_body = table.find_element(By.TAG_NAME,"tbody")
table_rows = table_body.find_elements(By.TAG_NAME,"tr")

In [298]:
#print(table_body.text)

In [299]:
for row in table_rows:
    table_data = row.find_elements(By.TAG_NAME, "td")
    row_data=[]
    for data in table_data:
        row_data.append(data.text)
    #print(row_data)
    

In [300]:
#SELECT REGION

select_element = Select(driver.find_element("name","region"))
select_element.select_by_value("europe") # Change here

In [301]:
#SELECT TYPE OF DATA usually is TEXT:LIST
select_element = Select(driver.find_element("name","TYPE"))
select_element.select_by_value("TEXT:LIST")

In [302]:
#SELECT YEAR
year="2024" #Change here
select_element = Select(driver.find_element("name","YEAR"))
select_element.select_by_visible_text(year)


In [303]:
#SELECT MONTH
month="11" #Change here
select_element = Select(driver.find_element("name","MONTH"))
select_element.select_by_value(month)

In [304]:
#SELECT FROM
day_from = "01" #Change here
utc_from= "00" #Change here
select_element = Select(driver.find_element("name","FROM"))
select_element.select_by_value(day_from+utc_from) # for ex. the month of the day is 10 and 
                                       # the time of the radiosonde gone is  00 UTC then value is 1000

In [305]:
#SELECT TO
day_to="05" #Change here
utc_to="00" #Change here
select_element = Select(driver.find_element("name","TO"))
select_element.select_by_value(day_to+utc_to) # Same as From

In [306]:
#CLICK ON THE IMAGE and SELECT STATION
station_id = '17064'  # Change Here

xpath = f"//area[contains(@href, \"g('{station_id}')\")]"
try:
    element = driver.find_element(By.XPATH, xpath)
    element.click()
    print(f"Element found for {station_id} : {element.get_attribute('outerHTML')}")
except:
    print(f"Element cannot found for {station_id} .")


Element found for 17064 : <area coords="390,268,5" shape="CIRCLE" href="javascript:g('17064')" title="17064  Istanbul/Kartal">


In [307]:
all_tabs = driver.window_handles

driver.switch_to.window(all_tabs[-1])


page_source = driver.page_source
#print(page_source)

In [308]:
from bs4 import BeautifulSoup as bs
import re

In [309]:
soup = bs(page_source,"html.parser")

In [310]:
#print(soup)

In [311]:
indices= soup.find_all(string=re.compile(r"Station number:"))

In [312]:
import pandas as pd

In [313]:
data = []
for i in range(0,len(indices)):
    for word in indices[i].split("\n"):
        b = word.strip()
        #print(b)
        data.append(b)
    data.remove('')
        

#print(data)




In [314]:
rowss=[]
for i in range(0,len(data)):
    column_name = data[i].split(":")[0]
    if ((data[i].split(":")[0] != "Observation time") and (data[i].split(":")[0] != "Station number")) :
        value=float(data[i].split(":")[1])
    else:
        value=data[i].split(":")[1]
    rowss.append({column_name:value})

In [315]:
#print(rowss)

In [330]:
result = {}
for d in rowss:
    for key, value in d.items():
        if key in result:
            if isinstance(result[key], list):
                result[key].append(value)
            else:
                result[key] = [result[key], value]
        else:
            result[key] = [value]
print(result)

{'Station number': [' 17064', ' 17064', ' 17064', ' 17064', ' 17064', ' 17064', ' 17064', ' 17064', ' 17064'], 'Observation time': [' 241101/0000', ' 241101/1200', ' 241102/0000', ' 241102/1200', ' 241103/0000', ' 241103/1200', ' 241104/0000', ' 241104/1200', ' 241105/0000'], 'Station latitude': [40.91, 40.91, 40.91, 40.91, 40.91, 40.91, 40.91, 40.91, 40.91], 'Station longitude': [29.16, 29.16, 29.16, 29.16, 29.16, 29.16, 29.16, 29.16, 29.16], 'Station elevation': [19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0], 'Showalter index': [8.43, 8.72, 5.39, 8.04, 6.29, 13.32, 13.07, 17.38, 13.95], 'Lifted index': [7.65, 5.84, 5.48, 6.21, 3.49, 12.69, 10.98, 13.16, 12.77], 'LIFT computed using virtual temperature': [7.59, 5.84, 5.37, 6.09, 3.43, 12.75, 11.03, 13.06, 12.75], 'SWEAT index': [64.19, 58.6, 75.62, 45.01, 79.18, 104.0, 59.01, 65.02, 59.01], 'K index': [13.3, -5.5, 7.1, 1.9, 13.9, 10.1, -9.2, -32.9, 5.9], 'Cross totals index': [15.7, 15.7, 17.3, 12.5, 19.1, 12.9, 15.1, -23.1, 1

In [317]:
import pandas as pd

In [336]:
max_length = max(len(v) for v in result.values())
for key in result:
    while len(result[key]) < max_length:
        result[key].append(None)

print(result)

the_indices = pd.DataFrame.from_dict(result)


{'Station number': [' 17064', ' 17064', ' 17064', ' 17064', ' 17064', ' 17064', ' 17064', ' 17064', ' 17064'], 'Observation time': [' 241101/0000', ' 241101/1200', ' 241102/0000', ' 241102/1200', ' 241103/0000', ' 241103/1200', ' 241104/0000', ' 241104/1200', ' 241105/0000'], 'Station latitude': [40.91, 40.91, 40.91, 40.91, 40.91, 40.91, 40.91, 40.91, 40.91], 'Station longitude': [29.16, 29.16, 29.16, 29.16, 29.16, 29.16, 29.16, 29.16, 29.16], 'Station elevation': [19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0], 'Showalter index': [8.43, 8.72, 5.39, 8.04, 6.29, 13.32, 13.07, 17.38, 13.95], 'Lifted index': [7.65, 5.84, 5.48, 6.21, 3.49, 12.69, 10.98, 13.16, 12.77], 'LIFT computed using virtual temperature': [7.59, 5.84, 5.37, 6.09, 3.43, 12.75, 11.03, 13.06, 12.75], 'SWEAT index': [64.19, 58.6, 75.62, 45.01, 79.18, 104.0, 59.01, 65.02, 59.01], 'K index': [13.3, -5.5, 7.1, 1.9, 13.9, 10.1, -9.2, -32.9, 5.9], 'Cross totals index': [15.7, 15.7, 17.3, 12.5, 19.1, 12.9, 15.1, -23.1, 1

In [341]:
first_date = day_from+"_"+month+"_"+year
last_date = day_to+"_"+month+"_"+year
the_indices.to_csv(first_date+"-"+last_date+"-"+station_id+".csv")